In [1]:
import sys
sys.path

['c:\\Users\\Ayden\\OneDrive - Monash University\\Uni\\2022 S2\\ETC3555\\electricity_models_3555\\analysis',
 'c:\\Users\\Ayden\\miniconda3\\python39.zip',
 'c:\\Users\\Ayden\\miniconda3\\DLLs',
 'c:\\Users\\Ayden\\miniconda3\\lib',
 'c:\\Users\\Ayden\\miniconda3',
 '',
 'c:\\Users\\Ayden\\miniconda3\\lib\\site-packages',
 'c:\\Users\\Ayden\\miniconda3\\lib\\site-packages\\win32',
 'c:\\Users\\Ayden\\miniconda3\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\Ayden\\miniconda3\\lib\\site-packages\\Pythonwin']

In [2]:
import tensorflow as tf

In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]